In [48]:
import json
import sys
import os
import numpy as np
from urllib import request
from torch.autograd import Variable
import time
import torch
import torch.optim as optim

## Fetch data, declare classes

In [2]:
def parse_info(url,file_name):
    head = {}
    head['User-Agent'] = 'Mozilla/5.0 (Linux; Android 4.1.1; Nexus 7 Build/JRO03D) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.166  Safari/535.19'
    req = request.Request(url, headers=head)
    response = request.urlopen(req)
    with open(file_name, "wb") as f:
        f.write(response.read())

In [3]:
# listen record
with open('./testData/myRecord.json') as record:
    record_json = json.load(record)['allData']
    
#friends id, top100 music input userid    
# follow list
with open('./testData/myFollows.json') as follows:
    follows_json = json.load(follows)['follow']
    
with open('./testData/detail.json') as user:
    user_json = json.load(user)['profile']

In [4]:
for i in range(len(record_json)):
    print(record_json[i]['score'], record_json[i]['song']['name'], record_json[i]['song']['pop'], record_json[i]['song']['al']['picUrl'])

100 Holy Wars...The Punishment Due 25.0 http://p4.music.126.net/APUBHGKPodtQ5rMN-YlHWg==/6636652185938189.jpg
95 Angry Again 50.0 http://p4.music.126.net/IfVRAnZ24F18agcHHJbw6Q==/636617232493161.jpg
88 Friday I'm In Love 95.0 http://p4.music.126.net/6aW4AorlJaWUMkq7WR-LQw==/17830780068060122.jpg
84 Gem 85.0 http://p3.music.126.net/EfH4pPT6AbWIWunyU78d6g==/6630055116402221.jpg
84 Symphony of Destruction 60.0 http://p3.music.126.net/w5bB_0ZsnTDb1GGiy53FyA==/6654244371511859.jpg
82 Paranoid 75.0 http://p4.music.126.net/V4Iexk29_FYhV6e_Z8iM_g==/2532175279150044.jpg
82 Soothsayer (dedicated to Aunt Suzie) 90.0 http://p4.music.126.net/AVd8EAEBwAsVamqv_LjqNQ==/6632254138952435.jpg
78 They Will Return 65.0 http://p4.music.126.net/sgziVnLsMuxxwM-0TD1NQQ==/2531075767438385.jpg
76 Master Of Puppets 90.0 http://p4.music.126.net/o5TGtGb2BxJo1yIllQP4JQ==/6621259023042637.jpg
74 Sultans Of Swing 25.0 http://p3.music.126.net/nxY7xELzxHsGByKdE0XK2Q==/17854969323878329.jpg
72 Kill the King 5.0 http://p3

In [5]:
record_json[0]['song']['al']['picUrl']

'http://p4.music.126.net/APUBHGKPodtQ5rMN-YlHWg==/6636652185938189.jpg'

In [6]:
class User():
    def __init__(self, user_id, user_name, followeds, follows, playlistCount):
        self.user_id = user_id
        self.user_name = user_name
        self.followed_num = followeds
        self.follows_num = follows
        self.playlist_num = playlistCount
        self.follows_list = []
    
    def getUserId(self):
        return self.user_id
    
    def getUserName(self):
        return self.user_name
    
    def getFollowedNum(self):
        return self.followed_num
    
    def getFollowsNum(self):
        return self.follows_num
    
    def getPlaylistNum(self):
        return self.playlist_num
    
    def getFollowsList(self):
        return self.follows_list
    
    def setUserId(self, user_id):
        self.user_id = user_id
    
    def setUserName(self, user_name):
        self.user_name = user_name
    
    def setFollowedNum(self, followed_num):
        self.followed_num = followed_num
    
    def setFollowsNum(self, follows_num):
        self.follows_num = follows_num
    
    def setPlaylistNum(self, playlist_num):
        self.playlist_num = playlist_num
        
    def setFollowsList(self, follows_list):
        self.follows_list = follows_list

In [7]:
class Music():
    def __init__(self, song_id, song_name, score, pop, url):
        self.song_id = song_id
        self.song_name = song_name
        self.score = score
        self.pop = pop
        self.url = url
        
    def getSongId(self):
        return self.song_id
    
    def getSongName(self):
        return self.song_name
    
    def getPop(self):
        return self.score
    
    def getSongUrl(self):
        return self.url
    
    def getScore(self):
        return self.score
    
    def setScore(self, score):
        self.score = score
        
    def setSongUrl(self, url):
        self.url= url
        
    def setSongId(self, song_id): 
        self.song_id = song_id
    
    def setSongName(self, song_name):
        self.song_name = song_name
        
    def setPop(self, score):
        self.score = score

In [8]:
# get a user's follows list
# input: uId
# output: followsId(list of ids), userList(List of User objs) 
def getFollows(uId):
    followsId = []
    userList = []
    filepath = './trainData/FollowsInfo/'+uId+'_follows.json'
    if(not os.path.exists(filepath)):
        parse_info('http://localhost:3000/user/follows?uid='+uId, filepath)
    with open(filepath) as follows:
        follows_json = json.load(follows)['follow']
    for i in range (len(follows_json)):
        user_id = str(follows_json[i]['userId'])
        user_name = follows_json[i]['nickname']
        followeds = follows_json[i]['followeds']
        follows = follows_json[i]['follows']
        playlistCount = follows_json[i]['playlistCount']
        followsId.append(user_id)
        userList.append(User(user_id, user_name, followeds, follows, playlistCount))
        
    return followsId, userList


# get a user's record list
# input: uId
# output: songList(list of Music objs)
def getRecord(uId):
    songList = []
    filepath = './trainData/MusicInfo/'+uId+'_record.json'
    if(not os.path.exists(filepath)):
        parse_info('http://localhost:3000/user/record?uid='+uId+'&type=0',filepath)
    with open(filepath) as records:
        records = json.load(records)['allData']
    for i in range(len(records)):
        song_id = records[i]['song']['id']
        song_name = records[i]['song']['name']        
        score = records[i]['score']
        if('pop' in records[i]['song'].keys()):
            pop = records[i]['song']['pop']
        else:
            pop = 10
        
        if('picUrl' in records[i]['song']['al']):
            url = records[i]['song']['al']['picUrl']
        else:
            url = ''
        
        songList.append(Music(song_id, song_name, score, pop, url))
    return songList

# get a user's profile
# input: uId
# output: user
def getUserInfo(uId):
    filepath = './trainData/UserInfo/'+uId+'_user.json'
    if(not os.path.exists(filepath)):
        parse_info('http://localhost:3000/user/detail?uid='+uId, filepath)
    with open(filepath) as userInfo:
        user_json = json.load(userInfo)['profile']
    
    user_id = user_json['userId']
    user_name = user_json['nickname']
    followeds = user_json['followeds']
    follows = user_json['follows']
    playlistCount = user_json['playlistCount']
    
    user = User(user_id, user_name, followeds, follows, playlistCount)
        
    return user
    
followsId, userList = getFollows('40356946')
# songList = getRecord('40356946')
# user_me = getUserInfo('40356946')

### get users' info

In [9]:
import queue
# start from one user, find a certain number of users, find their record, filter out those with pop less than 10.
start_user_id = '40356946'
start_user = getUserInfo(start_user_id)
q = queue.Queue()

q.put(start_user_id)

user_dict = {start_user_id:start_user}

count = 0
while(len(user_dict) < 50000):
    cur_user_id = q.get()
    cur_user = user_dict[cur_user_id]
#     print(cur_user_id)
    try:
        followsIdList, userList = getFollows(cur_user_id)
    except ValueError:
        print('value error in this user\'s file!')
        continue
    
    cur_user.setFollowsList(followsIdList)
    
    count += 1
    if(count % 500 == 0):
        print('%d th query, %d users fetched'%(count, len(user_dict)))
    for (followsId, user) in zip(followsIdList, userList):
        if(followsId not in user_dict):
            user_dict[followsId] = user
            q.put(followsId)

500 th query, 5281 users fetched
value error in this user's file!
1000 th query, 10789 users fetched
1500 th query, 15253 users fetched
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value 

### get song info

In [20]:
i = 0
total_song_ls = []
for user in user_dict:
    try:
        song_ls = getRecord(user)
    except (ValueError, KeyError):
        continue
    i +=1
    total_song_ls.append(song_ls)
    if(i % 1000 == 0):
        print('%d th user'%i)

1000 th user
2000 th user
3000 th user
4000 th user
5000 th user
6000 th user
7000 th user
8000 th user
9000 th user
10000 th user
11000 th user
12000 th user
13000 th user
14000 th user
15000 th user
16000 th user
17000 th user
18000 th user
19000 th user
20000 th user
21000 th user
22000 th user
23000 th user
24000 th user
25000 th user
26000 th user
27000 th user
28000 th user
29000 th user
30000 th user
31000 th user
32000 th user
33000 th user


In [21]:
print(sum(len(l) for l in total_song_ls))

3114603


In [22]:
song_id_lists = []
all_song_objects = {}
for l in total_song_ls:
    temp = []
    for song in l:
        if(song.getPop() < 25 or song.getScore() < 20):
            continue
        if song.getSongId() not in all_song_objects.keys():
            all_song_objects[song.getSongId()] = song
        temp.append(song.getSongId())
    song_id_lists.append(temp)


del(total_song_ls)

In [23]:
song_lists_size = len(song_id_lists)
songs_size = len(all_song_objects)
print('%d users with song lists'%song_lists_size)
print('%d songs in total'%songs_size)

33888 users with song lists
284076 songs in total


## Training

### co-occurence matrix

In [24]:
sid2num = {sid:num for num, sid in enumerate(all_song_objects.keys())}
num2sid = {num:sid for num, sid in enumerate(all_song_objects.keys())}

In [58]:
comat = {}
for i in range(song_lists_size):
    if((i+1)%2000 == 0):
        print('%d th list'%(i+1))
    for j in range(len(song_id_lists[i])):
        num = sid2num[song_id_lists[i][j]]
        for k in range(len(song_id_lists[i])):
            if(k == j):
                continue
            else:
                pnum = sid2num[song_id_lists[i][k]]
                if((num, pnum) in comat.keys()):
                    comat[(num, pnum)] += 1.0
                else:
                    comat[(num, pnum)] = 1.0
                    
comat = list(comat.items())

2000 th list
4000 th list
6000 th list
8000 th list
10000 th list
12000 th list
14000 th list
16000 th list
18000 th list
20000 th list
22000 th list
24000 th list
26000 th list
28000 th list
30000 th list
32000 th list


In [64]:
comat = sorted(comat, key=lambda x: x[1])[::-1]

In [60]:
print(all_song_objects[num2sid[6588]].getSongName())

如果我爱你


In [61]:
# declare hyper-parameters for training
embed_size = 50
batch_size = 10
l_rate = 0.005
num_epochs = 20
log_iter = 100

In [79]:
# filter out co-occurence that's too small
comat = [(song_pair, freq) for song_pair, freq in comat if freq > 2]

In [83]:
# update the all_song_objects dict to final_dict_objects
training_songs = set()
for item in comat:
    training_songs.add(item[0][0])
    training_songs.add(item[0][1])
    
final_song_objects = {}
for sid in all_song_objects.keys():
    if(sid2num[sid] in training_songs):
        final_song_objects[sid] = all_song_objects[sid]
        
del(all_song_objects)
songs_size = len(final_song_objects)

In [84]:
print(songs_size)

42157


### start training

In [49]:
alpha = 0.75
xmax = 100
# Weight function
def wf(x):
    if x < xmax:
        return (x/xmax)**alpha
    return 1

# Set up word vectors and biases
l_embed, r_embed = [
    [Variable(torch.from_numpy(np.random.normal(0, 0.01, (embed_size, 1))),
        requires_grad = True) for j in range(songs_size)] for i in range(2)]
l_biases, r_biases = [
    [Variable(torch.from_numpy(np.random.normal(0, 0.01, 1)), 
        requires_grad = True) for j in range(songs_size)] for i in range(2)]

# Set up optimizer
optimizer = optim.Adam(l_embed + r_embed + l_biases + r_biases, lr = l_rate)

# Batch sampling function
def gen_batch():
    sample = np.random.choice(np.arange(len(comat)), size=batch_size, replace=False)
    l_vecs, r_vecs, covals, l_v_bias, r_v_bias = [], [], [], [], []
    for chosen in sample:
        ind = comat[chosen][0]
        l_vecs.append(l_embed[ind[0]])
        r_vecs.append(r_embed[ind[1]])
        covals.append(comat[chosen][1])
        l_v_bias.append(l_biases[ind[0]])
        r_v_bias.append(r_biases[ind[1]])
    return l_vecs, r_vecs, covals, l_v_bias, r_v_bias

In [ ]:
# Train model
for epoch in range(num_epochs):
    num_batches = int(songs_list/batch_size)
    avg_loss = 0.0
    for batch in range(num_batches):
        optimizer.zero_grad()
        l_vecs, r_vecs, covals, l_v_bias, r_v_bias = gen_batch()
        loss = sum([torch.mul( ( torch.dot(l_vecs[i].view(-1), r_vecs[i].view(-1)) +
                l_v_bias[i] + r_v_bias[i] - np.log(covals[i]) )**2,
                wf(covals[i]) ) for i in range(batch_size)])
        avg_loss += loss.data[0]/num_batches
        loss.backward()
        optimizer.step()
    if batch % 1 == 0:
        print('Epoch [%d/%d], Iteration [%d/%d], Loss: %.4f'
              %(epoch, num_epochs, batch, num_batches, avg_loss)) 